## preprocessing of Can Tho Dataset and compariosn with HCMC dataset
AIm: Valdiate that both datasets dont differ too much from each other, so that they can be compared with each other


In [272]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""Data preprocessing for Can Tho survey dataset"""

__author__ = "Anna Buch, Heidelberg University"
__email__ = "a.buch@stud.uni-heidelberg.de"


import sys
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# sys.path.insert(0, "../")
# import utils.settings as s

# #s.init()
# seed = s.seed

import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('use_inf_as_na', True)
plt.figure(figsize=(20, 10))



<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

In [335]:
## load DS for flood event from 2011 in CanTHo
#df_cantho = pd.read_excel("../input_survey_data/input_data_cantho_2013_quicktest.xlsx", header=0) # only pre-selected features
df_cantho_raw = pd.read_excel("../input_survey_data/input_data_cantho_2013_raw_no_multiheader.xlsx") # removed mulitple header rows

# ## load HCMC DS for flood events between 2010-2020
df_hcmc_rloss = pd.read_excel("../input_survey_data/input_data_contentloss_tueb.xlsx")
df_hcmc_bred = pd.read_excel("../input_survey_data/input_data_businessreduction_tueb.xlsx")


In [274]:
df_cantho = df_cantho_raw

## missing values
df_cantho = df_cantho.replace({99.0: np.nan, 88.0: np.nan})

## rename columns 
rename cols to the ones used in HCMC ds

In [275]:

## assigne new columns which have to be created in this script, eg relative contnet loss
df_cantho["Target_relative_contentloss_euro"] = None
df_cantho["Target_contentloss_euro"] = None



col_names = {
       r'3.31$' : 'Target_eloss_VND', 
       r'^3.34$' : 'Target_gloss_VND', 
       r'3.40$' : 'Target_businessreduction',

       r'3.13.b$' : 'inundation_duration_day/month',
       r'3.13.c$' :'inundation_duration_hour/day',
       r'^3.12$' : 'water_depth_cm', 
       r'3.16$' : 'flowvelocity', 
       r'3.14$' : 'contaminations', 
       r'^2.2$' : 'flood_experience', 

       '3.7.' : 'emergency_measures', 
       '4.1.' : 'precautionary_measures',  
       # not incl pumping euqipmentn and saving of valuables, 
       # but incl cheap funriture and low-value usage

       r'7.3$' : 'bage', 
       r'11.8$' : 'b_area',
       r'11.9$' : 'floors',
       r'7.4$' : 'building_value_cat',
       '3.18.' : 'overall_problem_house',
       r'11.6$' : 'content_value_g_VND',
       r'11.7$' : 'content_value_e_VND',

       r'11.2$' : 'shp_employees', 
       r'11.3.1$' : 'shp_avgmonthly_sale_VND', 
       r'10.5$' : 'hh_monthly_income_cat',
       r'11.5$' : 'shp_registered_capital_VND_cat', # 11.5.	How much did you pay for opening your shop

       # '6.1' : 'perception_govern_management',
       # '6.2' : 'perception_govern_increase_management',
       #'6.3' : 'perception_govern_protection', # is scaled 1-6
       #'resilience', 
       #'resilienceLeftAlone',

       ## further variables which might be intresting but not needed for feature selection
       r'^1.4$' : 'shp_sector',
       r'^3.2$' : 'flood_type',
       r'^3.5$' : 'warning_time_day',  # for HCMC Q1P2.9 wanring_time_hours
       r'3.9$' : 'effect_emergency_measures',
       r'3.32$' : 'replaced_cost_e',
       r'3.34$' : 'replaced_cost_g',
       r'^5.1$' : 'risk_future_flood',
       r'^5.2$' : 'risk_consequents_future_flood',
       r'4.9.1$' : 'perception_too_destructive_floods',
       r'6.1.$' : 'flood_management',
       r'7.2$' : 'ownership',
       r'11.15$' : 'builing_elevation',
       'Type of house' : 'builing_type',
}

for k, v in col_names.items():
    df_cantho.rename(columns ={ i: re.sub(k, v, i) for i in  df_cantho.columns }, inplace=True )

## drop unneeded columns 
df_cantho = df_cantho[df_cantho.columns[ ~df_cantho.columns.str.match('^\d')]]


# target vars for relative and absolute costs on content loss [VND]
df_cantho.insert(0, "Target_eloss_VND", df_cantho.pop("Target_eloss_VND")) 
df_cantho.insert(1, "Target_gloss_VND", df_cantho.pop("Target_gloss_VND"))
df_cantho.insert(2, "Target_contentloss_euro", df_cantho.pop("Target_contentloss_euro"))
df_cantho.insert(3, "Target_relative_contentloss_euro", df_cantho.pop("Target_relative_contentloss_euro"))

# explanatory var: monthly reduction of business [%] 
df_cantho.insert(4, "Target_businessreduction", df_cantho.pop("Target_businessreduction"))  


df_cantho.columns[:]

Index(['Target_eloss_VND', 'Target_gloss_VND', 'Target_contentloss_euro',
       'Target_relative_contentloss_euro', 'Target_businessreduction',
       'shp_sector', 'flood_experience', 'flood_type', 'warning_time_day',
       'emergency_measures1', 'emergency_measures2', 'emergency_measures3',
       'emergency_measures4', 'emergency_measures5', 'emergency_measures6',
       'emergency_measures7', 'emergency_measures8', 'emergency_measures9',
       'effect_emergency_measures', 'water_depth_cm',
       'inundation_duration_day/month', 'inundation_duration_hour/day',
       'contaminations', 'flowvelocity', 'overall_problem_house1',
       'overall_problem_house2', 'overall_problem_house3',
       'overall_problem_house4', 'replaced_cost_e', 'precautionary_measures1',
       'precautionary_measures2', 'precautionary_measures3',
       'precautionary_measures4', 'precautionary_measures5',
       'precautionary_measures6', 'precautionary_measures7',
       'precautionary_measures8', 'pre

### Absolute content loss

In [276]:
## merge both absolute content loss targets
df_cantho["Target_contentloss_VND"]  = df_cantho.Target_eloss_VND + df_cantho.Target_gloss_VND
np.round(df_cantho["Target_contentloss_VND"], 2).describe()

## merge content values 
df_cantho["shp_content_value_VND"]  = df_cantho.content_value_g_VND + df_cantho.content_value_e_VND

df_cantho.drop(["Target_eloss_VND", "Target_gloss_VND", "content_value_g_VND", "content_value_e_VND"], axis=1, inplace=True)


### Hydrological variables


In [277]:
df_cantho["inundation_duration_h"] =  df_cantho["inundation_duration_hour/day"] # <<-- more similar to HCMC, than df_cantho["inundation_duration_day/month"] * inundation_duration_hour/day


## verify with HCMC
print("CanTHo", df_cantho.inundation_duration_h.describe(), "\n", "HCMC", df_hcmc_rloss.inundation_duration_h.describe())

# # water depth [cm]
print("CanTHo", df_cantho.water_depth_cm.describe(), "\n", "HCMC", df_hcmc_rloss.water_depth_cm.describe())
# median: 0.20cm, mean: 0.25m, max: 0.80m



CanTHo count   378.00
mean      4.33
std       3.40
min       1.00
25%       2.00
50%       4.00
75%       6.00
max      24.00
Name: inundation_duration_h, dtype: float64 
 HCMC count   380.00
mean     12.68
std      40.85
min       0.20
25%       2.00
50%       3.00
75%       6.00
max     600.00
Name: inundation_duration_h, dtype: float64
CanTHo count   378.00
mean     26.34
std      16.50
min       0.00
25%      12.75
50%      20.00
75%      40.00
max      80.00
Name: water_depth_cm, dtype: float64 
 HCMC count   388.00
mean     34.00
std      25.70
min       1.00
25%      15.00
50%      30.00
75%      50.00
max     150.00
Name: water_depth_cm, dtype: float64


In [278]:
## contamination
## rank contamination according to their occurence and damage potential: 0:no contamination, 1:light contamination and 2:heavy contamination

df_contaminations = df_cantho.filter(like="contaminations", axis=1)

df_cantho["contaminations"] = df_cantho["contaminations"].replace(
    {
    0.00 : 0, #  no contamination
    4.00 : 0, # no contamination  
    1.00 : 1, # light
    2.00 : 2,  # heavy
    3.00 : 2,  # heavy
    }
)
df_cantho.contaminations.value_counts()

contaminations
1.00    270
0.00    105
2.00      3
Name: count, dtype: int64

In [279]:
# assign according to HCMC distribution

df_cantho.flowvelocity = df_cantho.flowvelocity.replace(
    {
        1.0 : 8,    # once a year
        2.0 : 16,   # twice a year
        3.0 : 26,    # three times a year (doent exist for HCMC)
        4.0 : 36,   # 4 times a year
        5.0 : 76, 
    }
)
df_cantho.flood_experience.value_counts()


flood_experience
5.00    194
1.00    132
2.00     18
4.00     18
3.00     16
Name: count, dtype: int64

In [280]:
## flowvelo

## Paprotny 2021:  As for flow velocity, the respondents assessed i based on a qualitative scale, 
# providing a value from 1 to 6, with half-points possible (Thieken et al. 2005). 
# A value of 0.1 m/s was assigned to each full step of this qualitative scale. 


## flowvelocity high depends on the exepience and Einschätzung of the intrviewee
## e.g. has the interviewee ever seen turbulent water
## still the variable shows the general characteristics, if flood water is fast or not
## Extrapolation: use fv as input and also for damage-funs(due that maybe most important var in FI) 
## - flowvelo is highly related to inundation duration --> from inundation duration possible to derive flovelo 


df_cantho.flowvelocity = df_cantho.flowvelocity.replace(        # 1 - 6 # calm - torrential
    {
        1 : 0.1, 
        2 : 0.2, 
        3 : 0.3, 
        4 : 0.4, 
        5 : 0.5,
        6 : 0.6
    }
)

#df_cantho.flowvelocity.value_counts()
df_cantho.flowvelocity.describe()


count   378.00
mean     40.79
std      30.46
min       0.00
25%      16.00
50%      36.00
75%      76.00
max      76.00
Name: flowvelocity, dtype: float64

### Building variables

In [281]:
df_cantho["building_value_mVND"] = df_cantho["building_value_cat"].replace(
    {
        1.0 : 2.5,
        2.0 : 7.5,
        3.0 : 15,
        4.0 : 35,
        5.0 : 75,
        6.0 : 150,
        7.0 : 350,
        8.0 : 750,
        9.0 : 1500,
        10.0 : 3500,
    }
)
df_cantho = df_cantho.drop("building_value_cat", axis=1)


In [282]:
## bv
# median: 75 mVND, mean: 75 mVND, max 3500 mVND
print(df_cantho.floors.describe() ) # meaningless or only flat buildings (or count ground floor = 0, first level = 1 ?)
    
## fix floors :
## problably 0 : bungalow with only ground floor --> in HCMC would be this 1 floor
df_cantho["floors"] = df_cantho["floors"] +1 

## remove records with more than 3 floors
df_cantho = df_cantho.loc[df_cantho["floors"] <= 3, :]  # rm ~ 20 records



## verify with HCMC
print("CanTHo", df_cantho.floors.describe(), "\n", "HCMC: median and mean are at 2 floors ")



count   376.00
mean      0.61
std       0.90
min       0.00
25%       0.00
50%       0.00
75%       1.00
max       4.00
Name: floors, dtype: float64
CanTHo count   355.00
mean      1.46
std       0.68
min       1.00
25%       1.00
50%       1.00
75%       2.00
max       3.00
Name: floors, dtype: float64 
 HCMC: median and mean are at 2 floors 


### socio economic status
check if income , monthly sale etc is similar to the HCMC shophouses


In [283]:
df_cantho["hh_monthly_income_mVND"]  = df_cantho["hh_monthly_income_cat"].replace(
    {
        1.0 : 0.25,  # in mVND
        2.0 : 0.75,
        3.0 : 1.5,
        4.0 : 3.5,
        5.0 : 7.5,
        6.0 : 15,
        7.0 : 35,
        8.0 : 50,
    }
)

df_cantho.drop("hh_monthly_income_cat", axis=1, inplace=True)

In [284]:
# shp_registered_capital_VND

df_cantho["shp_registered_capital_mVND"]  = df_cantho["shp_registered_capital_VND_cat"].replace(
    {
        1.0 : 25,  # in mVND
        2.0 : 75,
        3.0 : 150,
        4.0 : 350,
        5.0 : 750,
        6.0 : 1500,
        7.0 : 3500,
    }
)

df_cantho.drop("shp_registered_capital_VND_cat", axis=1, inplace=True)
# df_cantho["shp_registered_capital_mVND"].describe()

## compared to HCMC
##  --> medians are similar, 1. and 3.Q: are also more or less similar when considering the categrical sturcutre for CanTho
## a bit too high values for CanTHo probably due to the categorical intervals were smallest registed captial value caputres most cases ,
# ## while for HCMC a continous scale was used in the questionary

In [285]:

df_cantho[["hh_monthly_income_mVND", "shp_employees", "shp_avgmonthly_sale_VND", "shp_registered_capital_mVND"]].describe()
# df_cantho[df_cantho.number_employees>=20]  # 5 businesses with more than 20 employees

## --> first keep also larger shops due that their absolute losses ae similar high aas for HCMC around 2500 €


,hh_monthly_income_mVND,shp_employees,shp_avgmonthly_sale_VND,shp_registered_capital_mVND
count,353.00,353.00,324.00,353.00
mean,6.73,2.05,14175925.93,77.05
std,7.44,3.14,48451182.93,296.99
min,0.25,1.00,300000.00,25.00
25%,3.50,1.00,2000000.00,25.00
50%,3.50,1.00,4000000.00,25.00
75%,7.50,2.00,10000000.00,25.00
max,50.00,37.00,600000000.00,3500.00


In [286]:
# df_cantho[df_cantho.number_employees>=20]  # 5 businesses with more than 20 employees

## --> first keep also larger shops due that their absolute losses ae similar high aas for HCMC around 2500 €

### Business-characteristics


In [287]:
## HCMC
# hh_income	[category]
# median 300 €, mean: 425 €, max: 3320 €

# number_employees	
# median: 2,  mean: 2,  max: 10

# shp_avg_monthly_sale
# median: 290 €, mean: 370 €, max: 2760 €



## Can Tho
# hh_income	[category]
# median: ~ 140€, mean 140€-380€, max > 2000€

# number_employees	
# median: 1 mean: 1,  max: 37

# shp_avg_monthly_sale [mVND]
# median: 192 €, mean: 1670 €, max: 346500 €

## --> surveyed people in CanTHo have smaller income than in HCMC --> TODO check ownership (more employee stauts than in HCMC ? )
## --> maybe remove shop in CanTho with many employees and high sale


## Indicators

In [288]:
## reset idnex to avoid problem when re-merging with indicators 
df_cantho = df_cantho.reset_index(drop=True)


In [289]:
## emergency

# df_cantho.emergency_measures1 - 9

## emergency measures as indicator : range: 0:no measures - 6: applied all measures 

pattern = [r"^emergency_measures.?"] 
pattern_cols = re.compile('|'.join(pattern))
df_emergency = df_cantho.filter(regex=pattern_cols, axis=1)
df_emergency

## create indicator as ratio between implemented and potentially implemented emergency measures
df_cantho["emergency_measures"] = None
df_cantho["emergency_measures"] = df_emergency.eq(1).sum(axis=1) / len(df_emergency.columns)

# keep only indicator in final df
df_cantho.drop(
    df_emergency.filter(
        regex=r"^(?:.+\d)$"
        ).columns, 
    axis=1, inplace=True
)
df_cantho.emergency_measures.describe()

count   355.00
mean      0.29
std       0.19
min       0.00
25%       0.11
50%       0.33
75%       0.44
max       0.89
Name: emergency_measures, dtype: float64

In [290]:
## overall_problem_house

## Cantho survey question
# 1: My house was collapsed or washed away 
# 2: My house was damage partly (specify)
# 3: My house was full of water which caused damage to the floor, walls, etc.


pattern = [r"overall_problem_house.(?<!4)$"] # get all columns but not overall_problem_house_r4: no problem/ problem 
pattern_cols = re.compile('|'.join(pattern))
df_problem_house = df_cantho.filter(regex=pattern_cols, axis=1)
df_problem_house.describe()

# ## create indicator (based on ranking scheme from HCMC)
df_cantho["overall_problem_house"] = None

idx = np.where(df_problem_house["overall_problem_house1"]==1)
df_cantho.overall_problem_house[idx[0].tolist()] = 6  # heavy building damage


idx = np.where(df_problem_house["overall_problem_house2"]==1)
df_cantho.overall_problem_house[idx[0].tolist()] = 4  # partly building damage

## fix typos in overall_problem_house3
df_problem_house["overall_problem_house3"] = df_problem_house["overall_problem_house3"].replace({10:1, 3:1})

idx = np.where(df_problem_house["overall_problem_house3"]==1)
df_cantho.overall_problem_house[idx[0].tolist()] = 3  # floor and wall damage


## keep only indicator in final df
df_cantho.drop(
    df_cantho.filter(regex=r"^overall_problem_house.$").columns,
    axis=1, inplace=True
)
print(df_cantho.columns)

Index(['Target_contentloss_euro', 'Target_relative_contentloss_euro',
       'Target_businessreduction', 'shp_sector', 'flood_experience',
       'flood_type', 'warning_time_day', 'effect_emergency_measures',
       'water_depth_cm', 'inundation_duration_day/month',
       'inundation_duration_hour/day', 'contaminations', 'flowvelocity',
       'replaced_cost_e', 'precautionary_measures1', 'precautionary_measures2',
       'precautionary_measures3', 'precautionary_measures4',
       'precautionary_measures5', 'precautionary_measures6',
       'precautionary_measures7', 'precautionary_measures8',
       'precautionary_measures9', 'precautionary_measures10',
       'perception_too_destructive_floods', 'risk_future_flood',
       'risk_consequents_future_flood', 'ownership', 'bage', 'shp_employees',
       'shp_avgmonthly_sale_VND', 'b_area', 'floors', 'builing_elevation',
       'builing_type', 'Direct_damage', 'Target_contentloss_VND',
       'shp_content_value_VND', 'inundation_duratio

In [291]:
# # # df_cantho.precautionary_measures1 - 10
# # #df_cantho.precautionary_measures1.value_counts()
# # df_precautionary.describe()

# pattern = [r"^precautionary_measures.*(?<![2,3])$"] # exclude information measures
# pattern_cols = re.compile('|'.join(pattern))
# df_precautionary = df_cantho.filter(regex=pattern_cols, axis=1)
# print(df_precautionary.columns)


In [292]:
## precaution measures 
# ## --> test first as seperate features to keep as much information as possible

pattern = [r"^precautionary_measures.*(?<![2,3])$"] # exclude information measures
pattern_cols = re.compile('|'.join(pattern))
df_precautionary = df_cantho.filter(regex=pattern_cols, axis=1)
# print(df_precautionary.columns)
df_precautionary.drop("precautionary_measures1", axis=1, inplace=True)


## fix typo
df_precautionary = df_precautionary.replace({40.00:np.nan})

# ## create ratio: 
df_precautionary = df_precautionary.replace(
    {
        4:1,
        3:1,    # impl before 
        2:1,    
        1:1,     
        0:0     # not at all  
    }
)
# ## devide into expensive and low cost precautionary measures due that expensive meausres seems to be better in flood-loss-reduction
df_precautionary_expensive = df_precautionary[["precautionary_measures9", "precautionary_measures10", "precautionary_measures7"]]
df_precautionary_low = df_precautionary.drop(["precautionary_measures9", "precautionary_measures10", "precautionary_measures7"], axis=1)



# create indicator as ratio between implemented and potentially implemented measures (in total 7 measures exist)
# range: 0.0: none measure were implemented before the flood event - 1.0: all potential measures were implemented before the reported flood
# --> splitting in low and expensive precaution measures reduces the importance of the feautres for the models remarkable
df_cantho["precautionary_measures_lowcost"] = None
df_cantho["precautionary_measures_lowcost"] = df_precautionary_low.sum(axis=1) / len(df_precautionary_low.columns)
df_cantho["precautionary_measures_expensive"] = None
df_cantho["precautionary_measures_expensive"] = df_precautionary_expensive.sum(axis=1) / len(df_precautionary_expensive.columns)


## keep only indicator in final df
df_cantho.drop(
    df_cantho.filter(regex=r"\d$").columns,
    # df_cantho.filter(regex=r"^precautionary_measures.$").columns,  # not caputere precuation_10
    axis=1, inplace=True
)
print(df_cantho.columns)


Index(['Target_contentloss_euro', 'Target_relative_contentloss_euro',
       'Target_businessreduction', 'shp_sector', 'flood_experience',
       'flood_type', 'warning_time_day', 'effect_emergency_measures',
       'water_depth_cm', 'inundation_duration_day/month',
       'inundation_duration_hour/day', 'contaminations', 'flowvelocity',
       'replaced_cost_e', 'perception_too_destructive_floods',
       'risk_future_flood', 'risk_consequents_future_flood', 'ownership',
       'bage', 'shp_employees', 'shp_avgmonthly_sale_VND', 'b_area', 'floors',
       'builing_elevation', 'builing_type', 'Direct_damage',
       'Target_contentloss_VND', 'shp_content_value_VND',
       'inundation_duration_h', 'building_value_mVND',
       'hh_monthly_income_mVND', 'shp_registered_capital_mVND',
       'emergency_measures', 'overall_problem_house',
       'precautionary_measures_lowcost', 'precautionary_measures_expensive'],
      dtype='object')


In [293]:
df_cantho[["precautionary_measures_lowcost", "precautionary_measures_expensive"]].describe()

## HCMC for both precautionary indicators
# keys stats are similar between both DS (similar mean and median ) 
## --> for CanTHo it seems that at least lesss often all types of low and expensive meausres were implemented compared to HCMC, but on average more than in HCMC (compare medians between both DS for low and expensive precautions)

,precautionary_measures_lowcost,precautionary_measures_expensive
count,355.00,355.00
mean,0.30,0.22
std,0.25,0.21
min,0.00,0.00
25%,0.00,0.00
50%,0.25,0.33
75%,0.50,0.33
max,1.00,0.67


In [294]:
# ## shp_registered_capital
# df_cantho["shp_registered_capital_mVND"].describe()

# ## HCMC
# ## 

In [295]:
# ## resilience as indicator:

# ## 1 : strong disagree - 5 : strong agree
# ## currently would leave at it is and see if its included in final Feature space
# ## TEST: as merged indcator and as seperate features
# ## o	Orderer rank okay as long its  quantitative

# ## select based on findings from PCA
# df_resilience = df[["resilience_city_protection", "resilience_govern_careing","resilience_neighbor_management"]]
# ## negatives loadings: resilience_govern_careing_increases
# print(df_resilience.columns)

# df["resilience"] = None
# df["resilience"] = df_resilience.sum(axis=1) / len(df_resilience.columns)

# ## rather pessimistic resilience variables
# df_resilience_leftalone = df[["resilience_left_alone", "resilience_more_future_affected"]]
# print(df_resilience_leftalone.columns)

# df["resilienceLeftAlone"] = None
# df["resilienceLeftAlone"] = df_resilience_leftalone.sum(axis=1) / len(df_resilience_leftalone.columns)

# ## --> potentiall resilinece_leftalone correlates positvely with target variables, 
# ## --> while the other resilience indicator may corellates negatively with the targets 

# ## keep only indicator in final df
# df.drop(
#     df.filter(like="resilience_").columns,
#     axis=1, inplace=True
# )
# print(df.columns)
# ## 1: disagree, 5: agree


In [296]:
# ## perception as indicator
# ## o	Orderer rank okay as long its  quantitative

# # Reduced /Porrer  -> 1
# # Maintained /Same  -> 2
# # Increased /Richer -> 3


# # ## based on findings from PCA
# # df_perception = df[["perception_govern_support_past", "perception_govern_support_future"]]
# # print(df_perception.columns)

# # df["perception"] = None
# # df["perception"] = df_perception.sum(axis=1) / len(df_perception.columns)

# ## keep only indicator in final df
# df.drop(
#     df.filter(like="perception_").columns, 
#     axis=1, inplace=True
# )



## Unify monetary values

In [297]:
# ## check for very small registered capital
# vars_money = df_cantho.filter(regex="_mVND", axis=1)
# try:
#     vars_money[vars_money.shp_registered_capital_mVND <=1.0]  # less than 40 euros
# except:
#     pass

*list of all monetary vars*

**Variables inflation corrected for 2020 (align with HCMC ds)**
<!-- - b_value_mVND	- price level for 2013 (year when survey was done)
- shp_building_value_mVND	- price level for 2013
- shp_content_value_mVND	- price level for 2013
- Target_contentloss_mVND	- price levels based on flood time
- shp_registered_capital_mVND  - price level for 2013
- hh_monthly_income_mVND     - continous [value ranges in mVND], # price level for 2013
- shp_avgmonthly_sale_mVND   - continous [value ranges in mVND], # price level for 2013 -->

**Variables inflation corrected for flood year**
Damage variables ('Target_eloss_VND', 'Target_gloss_VND') need to be inflation corrected based on flood time which was 2011

cpi_2020 = 168.8    ,  2020 = year when the survey was done



In [298]:
## check for very small registered capital
vars_money = df_cantho.filter(regex="_mVND", axis=1)


## covnert all columns with million VND --> VND

vars_money = np.where( (vars_money.values != np.nan),
            vars_money.values * 1000000, # convert to VND
            vars_money.values)

## rename columns
new_cols = df_cantho.filter(regex="_mVND", axis=1).columns.str.replace("_mVND", "_VND")
vars_money = pd.DataFrame(vars_money, columns=new_cols)
print(vars_money.columns)

df_cantho.drop( df_cantho.filter(regex="_mVND", axis=1).columns, axis=1, inplace=True)

Index(['building_value_VND', 'hh_monthly_income_VND',
       'shp_registered_capital_VND'],
      dtype='object')


In [299]:
## add remaining monetary variables
vars_money = pd.concat(
    [vars_money, 
    df_cantho[df_cantho.filter(regex="VND", axis=1).columns.tolist()]
    ], axis=1)
vars_money.columns  # sholuld all end with _VND (no _mVND)

Index(['building_value_VND', 'hh_monthly_income_VND',
       'shp_registered_capital_VND', 'shp_avgmonthly_sale_VND',
       'Target_contentloss_VND', 'shp_content_value_VND'],
      dtype='object')

*Conversion of VND to euro (or US$)*

Based on JRC, p.8 and Paprotny2018, eg.p245
The reported damage values have been converted to Euro using the the exchange rate for the year 2013 (mean annual value)

*Source:* 
- www.oanda.com/currency/historical-rates
-  www.ecb.europa.eu/stats/exchange/eurofxref/html/eurofxref-graph-idr.en.html


In [300]:
# price_index_year_of_issue = {
#     2011: 121.41,
#     2013: 137.78,
#     2014: 142.88,
#     2015: 140.43,
#     2016: 142.98,
#     2017: 149.22,
#     2018: 154.63,
#     2019: 158.38,
#     2020: 160.70
# }

price_index_2020 = 160.70
price_index_year = np.full( vars_money.shape[0], 121.41, dtype=float) # 
# price_index_year = data_ip2["flood_year"].astype("Int64").map(price_index_year_of_issue)  # series of cpi for each year of flood event

exchange_rate = 1 / 27155  #  dong-> euro in 2020 # 3.683e-05
# ## (based on eurostat: https://ec.europa.eu/eurostat/databrowser/view/ERT_BIL_EUR_A/default/line?lang=en )

*Inflation correction via GDP-deflator*


\begin{equation}
\begin{align*}

&uninflated_{2020} = losses_y * exchangerate_{2020} \\
&inflationrate = uninflated_{2020} * pindex_{2020} / pindex_y\\

\end{align*}
\end{equation}

where:
- losses_y : losses in VND for year y
- uninflated_{2020} : uninflated losses in euro for 2020
- exchangerate_{2020} : exchang erate for VND to euro in year 20202 
- pindex_{2020} : price index from GDP-deflator for 2020 
- pindex_y : price index from GDP-deflator in year y

Given that inflation is the percentage change in the overall price of an item in an economy, we can use the GDP deflator to calculate the inflation rate since its a measure of the price level.


*Further sources* \
*Paprotny 2018*: also used country-level GDP deflators for adjusting nominal to real losses in 2011 prices , p153, p244 \
*Sairam et al. 2020*


In [301]:
## GDP-deflator

vars_money["Target_contentloss_VND_gdp"] = None

##  only direct losses needs inflation correction in respect to flood time
for r in range(len(vars_money.Target_contentloss_VND)):
    ## exchange rate: convert VND in certain year to € in the same year
    uninflated_losses = (vars_money.Target_contentloss_VND[r] * exchange_rate) # get uninflated losses in euros for year 2020
    ## price index from GDP-deflator
    vars_money["Target_contentloss_VND_gdp"][r] = round(uninflated_losses * price_index_2020 / price_index_year[r], 1)



# ##  for all other monetary continous vars: only need exchange conversion
for c in vars_money.drop(["Target_contentloss_VND_gdp","Target_contentloss_VND"], axis=1).columns:
    vars_money[c] = vars_money[c].apply(pd.to_numeric)
    for r in range(len(vars_money[c])):
        ## convert VND_2020 to €_2020
        vars_money[c][r] = round((vars_money[c][r] * exchange_rate), 1)#.astype(int)



In [302]:

## rename columns
new_cols = vars_money.filter(regex="_VND", axis=1).columns.str.replace("_VND", "_euro")
vars_money.columns = new_cols
vars_money = vars_money.apply(pd.to_numeric)


In [303]:
# update all_input with unified and inflated currencies
df_cantho.drop(df_cantho.filter(regex=r"_mVND|_VND", axis=1).columns, axis=1, inplace=True) 
df_cantho = pd.concat([df_cantho, vars_money], axis=1)
df_cantho.drop("Target_contentloss_euro", axis=1, inplace=True)
df_cantho.rename(columns={"Target_contentloss_euro_gdp" : "Target_contentloss_euro"}, inplace=True)

df_cantho.filter(regex=r"euro|VND", axis=1).columns  # should be only euros


Index(['Target_relative_contentloss_euro', 'building_value_euro',
       'hh_monthly_income_euro', 'shp_registered_capital_euro',
       'shp_avgmonthly_sale_euro', 'shp_content_value_euro',
       'Target_contentloss_euro'],
      dtype='object')

## Targets
### absolute loss and Target business reduction 

In [304]:
# absolute content loss
df_cantho["Target_contentloss_euro"].describe()


# Target business reduction
df_cantho["Target_businessreduction"].describe()


## verify with HCMC
print("*CanTHo*", (df_cantho.Target_contentloss_euro).describe(), "\n", "*HCMC*: median: 0€ , mean:131€, 3.Q:78€, max:2600 €")
## --> abs losses are very similar in both DS
## -->  very similar in all keys-statistics  in min-max, median :-) interesting that fraction of zero-loss records is similar high as in HCMC

## verify with HCMC
print("*CanTHo*", df_cantho.Target_businessreduction.describe(), "\n", "*HCMC*", df_hcmc_bred.Target_businessreduction.describe())
## --> business reduction differ quite much: between avg < 10% in reduction in HCMC and 40% in Can THo 
## --> might not good to enrich bred ds with CanTHo

*CanTHo* count    354.00
mean      89.70
std      193.11
min        0.00
25%        0.00
50%        9.70
75%       73.10
max     1462.30
Name: Target_contentloss_euro, dtype: float64 
 *HCMC*: median: 0€ , mean:131€, 3.Q:78€, max:2600 €
*CanTHo* count   355.00
mean     40.58
std      22.09
min       0.00
25%      25.00
50%      40.00
75%      50.00
max     100.00
Name: Target_businessreduction, dtype: float64 
 *HCMC* count   353.00
mean     17.81
std      24.46
min       0.00
25%       0.00
50%      10.00
75%      30.00
max     100.00
Name: Target_businessreduction, dtype: float64


### content value
verify cv calculation for HCMC

In [305]:
## create content value

# print(df_cantho["shp_content_value_euro"].describe())


## Can Tho
# rloss : median: 0.01 , mean: max:
# CV: mean 3000 €, median: 300 € , max= 392 600 € (no inflation corrected)
print(df_cantho["shp_content_value_euro"].describe())  # in €

## make shp_cv as 1/4 of shp_bv
df_cantho["shp_content_value_euro_self"] = df_cantho["building_value_euro"] / df_cantho["floors"] * 0.25
df_cantho["shp_content_value_euro_self"]  =  df_cantho["shp_content_value_euro_self"] 
df_cantho["shp_content_value_euro_self"].describe() # in €

## cv survey question
# median: 260 € , mean: 2000 €, max: 57 000

## cv self (bv / floors * 0.25)
# median: 330 € , mean: 1100 €, max: 33 250

## HCMC euros (inflation corrected) : median: 3203 €,  mean: 4359 €, 1Q:257 €, 3Q: 4603 €, max:46032 €

#####################################################################

## HCMC after cleaning:
# abs closs : median 0, 3.Quantile= 80 €, mean 130 €, max 2600 €
# rcloss : median mean = 0.0, max = 0.3
# bred [%] : mean: 18%, median: 9 %

## Can Tho after cleaning: (no inflation)
# rloss : median: 0.0 , mean: 0.05,  max: 0.48
# CV: median: 310 € ,  mean 3340 €, max: 392.610 € (no inflation corrected)
## Bred : mean 40%, median 40%

## TODO 
# DONE - from can tho remove rloss > 0.8 --> gets similar to HCMC with max=0.3 ?
# CURR other method to derive / verify CV for HCMC  --> source why Logitic Regression performed so bad 
# Bred is quite different between both DS --> idea: apply can tho DS only to estimate closs fraction ?, do extrapoltion only for closs (not for bred)
 
## HCMC
# Target_contentloss_euro	shp_content_value_euro	Target_relative_contentloss_euro	Target_businessreduction
# count	371.0	324.0	315.0	350.0
# mean	131.6	4359.2	0.0	    17.4
# std	336.1	4047.6	0.1	    24.1
# min	0.0	    257.8	0.0	    0.0
# 25%	0.0	    1841.3	0.0	    0.0
# 50%	0.0	    3203.8	0.0	    8.5
# 75%	77.9	4603.2	0.0	    30.0
# max	2615.6	46032.0	1.0	    100.0



count     351.00
mean     1853.24
std      5569.73
min         0.00
25%        69.05
50%       221.00
75%       847.00
max     55238.40
Name: shp_content_value_euro, dtype: float64


count     354.00
mean     1063.56
std      2445.13
min         0.00
25%       138.10
50%       322.23
75%      1074.08
max     32222.42
Name: shp_content_value_euro_self, dtype: float64

In [306]:
df_cantho["shp_content_value_euro"].describe()

count     351.00
mean     1853.24
std      5569.73
min         0.00
25%        69.05
50%       221.00
75%       847.00
max     55238.40
Name: shp_content_value_euro, dtype: float64

### Target relative closs

In [307]:
## cv from survey
df_cantho["Target_relative_contentloss_euro"] = df_cantho["Target_contentloss_euro"] / df_cantho["shp_content_value_euro"]
# set all zero-loss cases to 0
df_cantho = df_cantho.apply(pd.to_numeric)
df_cantho.Target_relative_contentloss_euro[df_cantho.Target_relative_contentloss_euro.isna()] = 0.0


## self calc cv
df_cantho["Target_relative_contentloss_euro_self"] = df_cantho["Target_contentloss_euro"] / df_cantho["shp_content_value_euro_self"]
# set all zero-loss cases to 0
df_cantho = df_cantho.apply(pd.to_numeric)
df_cantho.Target_relative_contentloss_euro_self[df_cantho.Target_relative_contentloss_euro_self.isna()] = 0.0


In [308]:
## rloss > cv
print("Records with relative content loss > total content value :", sum(df_cantho.Target_relative_contentloss_euro > 1.0) )
print("Records with self-calc. relative content loss > total content value :", sum(df_cantho.Target_relative_contentloss_euro_self > 1.0) )

## drop these records where rloss > cv
df_cantho = df_cantho.loc[~(df_cantho.Target_relative_contentloss_euro >= 1.0), :]
df_cantho = df_cantho.loc[~(df_cantho.Target_relative_contentloss_euro_self >= 1.0), :]


# ## drop these records where rloss > cv
# df_cantho = df_cantho.loc[~(df_cantho.Target_rcloss >= 0.5), :]

df_cantho[["Target_contentloss_euro", "Target_relative_contentloss_euro", "Target_relative_contentloss_euro_self", 
           "shp_content_value_euro", "shp_content_value_euro_self", "Target_businessreduction"]].describe()


Records with relative content loss > total content value : 18
Records with self-calc. relative content loss > total content value : 28


,Target_contentloss_euro,Target_relative_contentloss_euro,Target_relative_contentloss_euro_self,shp_content_value_euro,shp_content_value_euro_self,Target_businessreduction
count,316.00,317.00,317.00,313.00,316.00,317.00
mean,57.71,0.10,0.11,1732.96,1154.96,40.47
std,130.07,0.19,0.20,5536.78,2568.93,21.54
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,73.70,138.10,25.00
50%,0.00,0.00,0.00,221.00,345.24,40.00
75%,48.70,0.10,0.12,810.20,1150.80,50.00
max,974.90,0.93,0.99,55238.40,32222.42,100.00


**cv self** is similar to the cv from questionary, therefore remove later one and the **respective Target version for loss ratio derived from it**, to have common approach of cv estimation in both DS (HCMC, CanTho)

In [309]:
df_cantho = df_cantho.drop(["Target_relative_contentloss_euro", "shp_content_value_euro"], axis=1)
df_cantho.rename(columns={
    "Target_relative_contentloss_euro_self" : "Target_relative_contentloss_euro",
    "shp_content_value_euro_self" : "shp_content_value_euro",
    }, inplace=True)


In [310]:
# ## remove shp sector # due that no expalnation which category revers to which sector
# df_cantho.shp_sector.value_counts()


## Explore variable's key statistics - are they similar to HCMC ?


In [311]:
df_cantho.columns

Index(['Target_businessreduction', 'shp_sector', 'flood_experience',
       'flood_type', 'warning_time_day', 'effect_emergency_measures',
       'water_depth_cm', 'inundation_duration_day/month',
       'inundation_duration_hour/day', 'contaminations', 'flowvelocity',
       'replaced_cost_e', 'perception_too_destructive_floods',
       'risk_future_flood', 'risk_consequents_future_flood', 'ownership',
       'bage', 'shp_employees', 'b_area', 'floors', 'builing_elevation',
       'builing_type', 'Direct_damage', 'inundation_duration_h',
       'emergency_measures', 'overall_problem_house',
       'precautionary_measures_lowcost', 'precautionary_measures_expensive',
       'building_value_euro', 'hh_monthly_income_euro',
       'shp_registered_capital_euro', 'shp_avgmonthly_sale_euro',
       'Target_contentloss_euro', 'shp_content_value_euro',
       'Target_relative_contentloss_euro'],
      dtype='object')

## Major cleaning 

Drop uneeded variables in final feature space

In [312]:
df_cantho = df_cantho.drop([
    'inundation_duration_day/month', 'inundation_duration_hour/day',
    'flood_type', 
    "shp_sector",  # have currently no information about category description
    "ownership",
    "Direct_damage", 
    #"No", "Year", "floors", "distance_to_river",
    ], axis=1
)
df_cantho.columns

Index(['Target_businessreduction', 'flood_experience', 'warning_time_day',
       'effect_emergency_measures', 'water_depth_cm', 'contaminations',
       'flowvelocity', 'replaced_cost_e', 'perception_too_destructive_floods',
       'risk_future_flood', 'risk_consequents_future_flood', 'bage',
       'shp_employees', 'b_area', 'floors', 'builing_elevation',
       'builing_type', 'inundation_duration_h', 'emergency_measures',
       'overall_problem_house', 'precautionary_measures_lowcost',
       'precautionary_measures_expensive', 'building_value_euro',
       'hh_monthly_income_euro', 'shp_registered_capital_euro',
       'shp_avgmonthly_sale_euro', 'Target_contentloss_euro',
       'shp_content_value_euro', 'Target_relative_contentloss_euro'],
      dtype='object')

In [313]:
# df_cantho.info()
# df_cantho.columns

### missing data

In [314]:
## delete features with more than 10% missing values
print("Percentage of missing values per feature [%]\n", round(df_cantho.isna().mean().sort_values(ascending=False)[:15]  * 100), 2) 


Percentage of missing values per feature [%]
 overall_problem_house              30.00
shp_avgmonthly_sale_euro            9.00
warning_time_day                    4.00
b_area                              1.00
replaced_cost_e                     1.00
shp_employees                       1.00
hh_monthly_income_euro              1.00
shp_registered_capital_euro         1.00
risk_future_flood                   0.00
building_value_euro                 0.00
bage                                0.00
Target_contentloss_euro             0.00
shp_content_value_euro              0.00
emergency_measures                  0.00
precautionary_measures_expensive    0.00
dtype: float64 2


### single dataset for bred and rloss

In [315]:
df_cantho_rloss = df_cantho
df_cantho_rloss = df_cantho_rloss.drop(
    [
    "Target_contentloss_euro",
    "Target_businessreduction",
    "shp_content_value_euro",
    ]
    , axis=1)
print(df_cantho_rloss.shape)
df_cantho_rloss.columns


(317, 26)


Index(['flood_experience', 'warning_time_day', 'effect_emergency_measures',
       'water_depth_cm', 'contaminations', 'flowvelocity', 'replaced_cost_e',
       'perception_too_destructive_floods', 'risk_future_flood',
       'risk_consequents_future_flood', 'bage', 'shp_employees', 'b_area',
       'floors', 'builing_elevation', 'builing_type', 'inundation_duration_h',
       'emergency_measures', 'overall_problem_house',
       'precautionary_measures_lowcost', 'precautionary_measures_expensive',
       'building_value_euro', 'hh_monthly_income_euro',
       'shp_registered_capital_euro', 'shp_avgmonthly_sale_euro',
       'Target_relative_contentloss_euro'],
      dtype='object')

In [316]:

df_cantho_bred = df_cantho
df_cantho_bred = df_cantho_bred.drop(
    [
    "Target_contentloss_euro",
    "Target_relative_contentloss_euro"
    ],
    axis=1)

print(df_cantho_bred.shape)
df_cantho_bred.columns

(317, 27)


Index(['Target_businessreduction', 'flood_experience', 'warning_time_day',
       'effect_emergency_measures', 'water_depth_cm', 'contaminations',
       'flowvelocity', 'replaced_cost_e', 'perception_too_destructive_floods',
       'risk_future_flood', 'risk_consequents_future_flood', 'bage',
       'shp_employees', 'b_area', 'floors', 'builing_elevation',
       'builing_type', 'inundation_duration_h', 'emergency_measures',
       'overall_problem_house', 'precautionary_measures_lowcost',
       'precautionary_measures_expensive', 'building_value_euro',
       'hh_monthly_income_euro', 'shp_registered_capital_euro',
       'shp_avgmonthly_sale_euro', 'shp_content_value_euro'],
      dtype='object')

## Merge with HCMC dataset

In [336]:
## make all DS as floats

#.info()
print(df_hcmc_bred.shape)

df_hcmc_bred.rename(columns={
    "shp_avgmonthly_sale": "shp_avgmonthly_sale_euro",
    "hh_monthly_income": "hh_monthly_income_euro"
}, inplace=True)


df_hcmc_rloss.rename(columns={
    "shp_avgmonthly_sale": "shp_avgmonthly_sale_euro",
    "hh_monthly_income": "hh_monthly_income_euro"
}, inplace=True)

print(df_hcmc_rloss.shape)
df_hcmc_rloss.columns


(393, 20)
(393, 18)


Index(['Target_relative_contentloss_euro', 'inundation_duration_h',
       'water_depth_cm', 'flowvelocity', 'flood_experience', 'bage', 'b_area',
       'shp_employees', 'shp_avgmonthly_sale_euro', 'hh_monthly_income_euro',
       'emergency_measures', 'precautionary_measures_lowcost',
       'precautionary_measures_expensive', 'resilience', 'resilienceLeftAlone',
       'contaminations', 'overall_problem_house',
       'shp_registered_capital_euro'],
      dtype='object')

In [338]:
df_cantho_rloss.columns

df_hcmc_rloss.drop("overall_problem_house", axis=1, inplace=True)

In [339]:
common_cols = list(set.intersection(*(set(df.columns) for df in [df_hcmc_rloss, df_cantho_rloss])))
df_rloss_joined = pd.concat([df[common_cols] for df in [df_hcmc_rloss, df_cantho_rloss]], ignore_index=True)
print(df_rloss_joined.shape)
df_rloss_joined.head(3)   # not containing HCMCs shp_sector, 'resilience', 'resilienceLeftAlone',

(710, 15)


,emergency_measures,bage,shp_avgmonthly_sale_euro,shp_registered_capital_euro,hh_monthly_income_euro,Target_relative_contentloss_euro,b_area,precautionary_measures_expensive,inundation_duration_h,flowvelocity,water_depth_cm,precautionary_measures_lowcost,flood_experience,contaminations,shp_employees
0,0.00,12.00,7.50,184.10,3.00,0.00,25.00,0.00,2.00,0.20,10.00,0.00,76.00,0.00,1.00
1,0.00,0.00,7.50,368.30,8.00,0.00,21.00,0.00,2.00,0.20,15.00,0.00,76.00,1.00,1.00
2,0.00,21.00,2.50,368.30,15.00,0.00,50.00,0.33,1.00,0.20,2.00,0.75,76.00,0.00,1.00


In [329]:
common_cols = list(set.intersection(*(set(df.columns) for df in [df_hcmc_bred, df_cantho_bred])))
df_bred_joined = pd.concat([df[common_cols] for df in [df_hcmc_bred, df_cantho_bred]], ignore_index=True)
print(df_bred_joined.shape)
df_bred_joined.head(3)   # not containing HCMCs shp_sector, 'resilience', 'resilienceLeftAlone',

(710, 17)


,emergency_measures,bage,shp_content_value_euro,shp_avgmonthly_sale_euro,shp_registered_capital_euro,hh_monthly_income_euro,b_area,precautionary_measures_expensive,inundation_duration_h,flowvelocity,Target_businessreduction,water_depth_cm,overall_problem_house,precautionary_measures_lowcost,flood_experience,contaminations,shp_employees
0,0.00,12.00,1841.30,7.50,184.10,3.00,25.00,0.00,2.00,0.20,0.00,10.00,5.00,0.00,76.00,0.00,1.00
1,0.00,0.00,1841.30,7.50,368.30,8.00,21.00,0.00,2.00,0.20,0.00,15.00,6.00,0.00,76.00,1.00,1.00
2,0.00,21.00,33143.10,2.50,368.30,15.00,50.00,0.33,1.00,0.20,0.00,2.00,1.00,0.75,76.00,0.00,1.00


In [219]:
df_hcmc_rloss.columns

Index(['Target_relative_contentloss_euro', 'inundation_duration_h',
       'water_depth_cm', 'flowvelocity', 'flood_experience', 'bage', 'b_area',
       'shp_employees', 'shp_avgmonthly_sale', 'hh_monthly_income',
       'emergency_measures', 'precautionary_measures_lowcost',
       'precautionary_measures_expensive', 'resilience', 'resilienceLeftAlone',
       'contaminations', 'overall_problem_house',
       'shp_registered_capital_euro'],
      dtype='object')

In [ ]:
# all_input_contentloss = all_input_contentloss.drop("Target_contentloss_euro", axis=1)
# all_input_contentloss = all_input_contentloss.drop(['shp_sector_1.0', 'shp_sector_2.0', 'shp_sector_3.0'], axis=1)
# all_input_contentloss = all_input_contentloss.drop(['flood_type.1', 'flood_type.2','flood_type.3'], axis=1)

# all_input_business_reduction = all_input_business_reduction.drop("Target_relative_contentloss_euro", axis=1)
# all_input_business_reduction = df_bred_joined.drop(['shp_sector' ,'flood_type.1', 'flood_type.2','flood_type.3'], axis=1)


In [341]:
## save to disk
df_rloss_joined.to_excel("../input_survey_data/input_data_contentloss_tueb_cantho.xlsx", index=False)
df_bred_joined.to_excel("../input_survey_data/input_data_businessreduction_tueb_cantho.xlsx", index=False)

In [342]:
print("Relative content loss dataset")
print("Number of candidate predictors ", df_rloss_joined.shape[1])
print("Number of cases ", df_rloss_joined.shape[0])

print("\nBusiness reduction dataset")
print("Number of candidate predictors ", df_bred_joined.shape[1])
print("Number of cases ", df_bred_joined.shape[0])

Relative content loss dataset
Number of candidate predictors  15
Number of cases  710

Business reduction dataset
Number of candidate predictors  17
Number of cases  710


## Left overs

In [222]:
## Target relative closs

# #t[["abs_closs_VND", "shp_content_value_VND"]] = t[["abs_closs_VND", "shp_content_value_VND"]].fillna(0, inplace=True)
# df_cantho["Target_rcloss"] = df_cantho["abs_closs_VND"] / df_cantho["shp_content_value_VND"]
# # set all zero-loss cases to 0
# df_cantho = df_cantho.apply(pd.to_numeric)
# df_cantho.Target_rcloss[df_cantho.Target_rcloss.isna()] = 0.0

## rloss > cv
print("Records with relative content loss exceding the content values for businesses:", sum(df_cantho.Target_rcloss > 1.0) )
# t.Target_rcloss[t.Target_rcloss > 1.0]  = 1.0
# print(all_input_contentloss[all_input_contentloss.Target_relative_contentloss_euro > 0.99 ])
df_cantho[["abs_closs_VND", "shp_content_value_VND", "Target_rcloss", "Target_bred"]].describe()

#df_cantho["Target_rcloss"] = df_cantho["abs_closs_VND"] / df_cantho["shp_content_value_VND"]
df_cantho["Target_rcloss"] = df_cantho["abs_closs_VND"] / df_cantho["shp_content_value_VND_self"]

# set all zero-loss cases to 0
df_cantho = df_cantho.apply(pd.to_numeric)
df_cantho.Target_rcloss[df_cantho.Target_rcloss.isna()] = 0.0

## rloss > cv
print("Records with relative content loss > total content value :", sum(df_cantho.Target_rcloss > 1.0) )

## drop these records where rloss > cv
df_cantho = df_cantho.loc[~(df_cantho.Target_rcloss >= 1.0), :]

df_cantho[["abs_closs_VND", "shp_content_value_VND_self", "Target_rcloss", "Target_bred"]].describe()


# Overestimation of CV for small businesses 
# --> businesses with overestimated Cv is charcterized by low number of employees

shps_with_overeestimated_cv = df_cantho.loc[df_cantho["shp_content_value_VND"]  <= 2000000.00, :]  # using 25% of busineeses with smallest CV [1.Qunatile]
shps_with_overeestimated_cv.number_employees.value_counts()  
## --> most of the shops with small cv are indeed very small businesses

# ## drop these records where rloss > cv
# df_cantho = df_cantho.loc[~(df_cantho.Target_rcloss >= 0.5), :]

# df_cantho[["abs_closs_VND", "shp_content_value_VND_self", "Target_rcloss", "Target_bred"]].describe()

# df_cantho.abs_closs_VND.describe()  # max abs loss is 2300 €

# ## explore cases where rloss > cv
# tt = t.loc[t.Target_rcloss > 1.0, :]
# tt.sort_values("abs_closs_VND", ascending=False)


## get rloss to similar ratio as for HCMC (rloss=0.3)
df_cantho =  df_cantho.loc[~(df_cantho.Target_rcloss >= 1.0), :] 
#df_cantho =  df_cantho.loc[~(df_cantho.Target_rcloss >= .5), :]   # removed ~ 15 records with higher loss ratio than 50%
df_cantho[["abs_closs_VND", "shp_content_value_VND", "Target_rcloss", "Target_bred"]].describe()

## Can Tho
# Abs Closs: median: 1 €, 3.Quantile: 50 €, mean 74 € , max:  2310 €
# rloss : median: 0.0 , mean: 0.05,  max: 0.48
# CV: mean 3340 €, median: 310 € , max: 392.610 € (no inflation corrected)
## Bred : mean 40%, median 40%

KeyError: 'abs_closs_VND'